In [1]:
!pip install keras

    100% |████████████████████████████████| 337kB 2.7MB/s ta 0:00:01
    100% |████████████████████████████████| 256kB 3.0MB/s ta 0:00:01
    100% |████████████████████████████████| 46.7MB 25kB/s  eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /home/aladetan/.cache/pip/wheels/03/05/65/bdc14f2c6e09e82ae3e0f13d021e1b6b2481437ea2f207df3f
Successfully built pyyaml
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
import json
import pandas as pd
import numpy as np

In [74]:
import spotipy
from spotipy import util
from config import config

In [77]:
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=config['spotify_id'], client_secret=config['spotify_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Quick tools

### check # of (all) labelled songs and features

In [24]:
l = pd.read_json('labelled', orient='rows')
f  = pd.read_json('features')
print(l.shape[1])
print(f.shape[0])
print l.shape[1] == f.shape[0]

181
181
True


## Data Collection

Features are already downloaded to save time/bandwith, but will continue to recollect data in notebook until data is too big/labelling is done

In [63]:
labelled = json.load(open('labelled'))

In [58]:
def mood_encode(moods):
    values = {'chill':0, 'hype':2, 'happy':1, 'sad':2}
#     chill happy 1
#     chill sad 2
#     hype happy 3
#     hype sad 4
    output = 0
    for mood in moods:
        output += values[mood]
        
    return output

In [65]:
sample = []
for row in labelled:
    label = mood_encode(labelled[row]['mood'])
    sample.append([row,labelled[row]['name'],label])
songs = pd.DataFrame(sample, columns=['id','title','label']).set_index('id')
songs.head()

,title,label
id,,
5ClmYd83GFMM8Unue3dqKJ,Bands,3
2ImWMn8Pqxa8gHfk1ZJvAh,Use My Heart,2
7gYwIAHB6VxzLJFSZMMv8i,I Forget Where We Were,2
2QmU4B8fsyWcTQLVDl06Pk,Spice,1
6X1g3qnbYDyaDq3nfuMXEQ,Bravado,2


In [66]:
features = pd.read_json('features').drop(columns=['analysis_url','track_href','uri','type']).set_index('id')
features.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,
3Ed55g8hCz1GmqRTelIPIW,0.8840,0.374,212853,0.2760,0.012300,7,0.1220,-14.945,1,0.0353,80.169,4,0.412
6J3YPOejivwLSjM2dru5Ad,0.0253,0.818,210675,0.5630,0.000000,2,0.0909,-8.096,1,0.3980,152.108,4,0.486
4h0zU3O9R5xzuTmNO7dNDU,0.9650,0.710,275280,0.0565,0.000000,7,0.0889,-15.099,1,0.0534,123.994,4,0.348
7CeJuYHjC7wmtwMWBDKcvV,0.3250,0.478,252293,0.5310,0.000048,0,0.1630,-7.436,0,0.0393,170.026,4,0.382
1yWpTxNpz61uFb2Y4He9Xe,0.0485,0.466,206480,0.6900,0.000000,8,0.2010,-5.186,1,0.0857,153.504,3,0.478


In [67]:
data = songs.join(features)
data.head()

,title,label,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,,
5ClmYd83GFMM8Unue3dqKJ,Bands,3,0.210,0.834,98734,0.457,0.00000,10,0.0641,-10.775,1,0.5720,158.008,4,0.5330
2ImWMn8Pqxa8gHfk1ZJvAh,Use My Heart,2,0.209,0.361,230253,0.658,0.00456,9,0.0887,-9.365,1,0.0400,82.979,4,0.1690
7gYwIAHB6VxzLJFSZMMv8i,I Forget Where We Were,2,0.437,0.299,281213,0.470,0.17600,0,0.0919,-9.542,1,0.0346,113.506,4,0.1820
2QmU4B8fsyWcTQLVDl06Pk,Spice,1,0.514,0.574,219648,0.451,0.00000,6,0.0823,-10.803,0,0.1910,83.950,4,0.6160
6X1g3qnbYDyaDq3nfuMXEQ,Bravado,2,0.526,0.493,221409,0.539,0.02280,7,0.1690,-10.329,1,0.0995,175.856,4,0.0384


Need to reshape df to numpy array for keras

In [68]:
array = data.drop(columns=['title']).values
x = array[:,1:]
y = array[:,0]
dimension = x.shape[1]
print(dimension)

13


## First NN Attempt

In [34]:
from keras.models import Sequential
from keras.layers import Dense
np.random.seed(7)

### Model

In [51]:
model = Sequential()
model.add(Dense(21, input_dim=dimension, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compling

In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit

In [69]:
model.fit(x, y, epochs=150, batch_size=10)

Epoch 1/150
181/181 [==============================] - 0s 715us/step - loss: -16.7351 - acc: 0.0497
Epoch 2/150
181/181 [==============================] - 0s 208us/step - loss: -16.7351 - acc: 0.0497
Epoch 3/150
181/181 [==============================] - 0s 211us/step - loss: -16.7351 - acc: 0.0497
Epoch 4/150
181/181 [==============================] - 0s 259us/step - loss: -16.7351 - acc: 0.0497
Epoch 5/150
181/181 [==============================] - 0s 256us/step - loss: -16.7351 - acc: 0.0497
Epoch 6/150
181/181 [==============================] - 0s 202us/step - loss: -16.7351 - acc: 0.0497
Epoch 7/150
181/181 [==============================] - 0s 262us/step - loss: -16.7351 - acc: 0.0497
Epoch 8/150
181/181 [==============================] - 0s 223us/step - loss: -16.7351 - acc: 0.0497
Epoch 9/150
181/181 [==============================] - 0s 192us/step - loss: -16.7351 - acc: 0.0497
Epoch 10/150
181/181 [==============================] - 0s 198us/step - loss: -16.7351 - acc: 0.0497

181/181 [==============================] - 0s 185us/step - loss: -16.7351 - acc: 0.0497
Epoch 83/150
181/181 [==============================] - 0s 187us/step - loss: -16.7351 - acc: 0.0497
Epoch 84/150
181/181 [==============================] - 0s 202us/step - loss: -16.7351 - acc: 0.0497
Epoch 85/150
181/181 [==============================] - 0s 176us/step - loss: -16.7351 - acc: 0.0497
Epoch 86/150
181/181 [==============================] - 0s 165us/step - loss: -16.7351 - acc: 0.0497
Epoch 87/150
181/181 [==============================] - 0s 214us/step - loss: -16.7351 - acc: 0.0497
Epoch 88/150
181/181 [==============================] - 0s 175us/step - loss: -16.7351 - acc: 0.0497
Epoch 89/150
181/181 [==============================] - 0s 198us/step - loss: -16.7351 - acc: 0.0497
Epoch 90/150
181/181 [==============================] - 0s 182us/step - loss: -16.7351 - acc: 0.0497
Epoch 91/150
181/181 [==============================] - 0s 168us/step - loss: -16.7351 - acc: 0.0497
Epo

### Evaluation

In [70]:
### need to split up testing and training data

In [73]:
scores = model.evaluate(x, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

181/181 [==============================] - 0s 42us/step

acc: 4.97%


### Predicting

Input a new song a get a prediction

In [80]:
spotify_song_id = '2KRQR1VK6OoJxpwVzwDv72' #funky duck by vulfpeck

In [90]:
spotify_song_id = '2gTYVoQCUh0QNUaFix01ld' #chun-li by nicki minaj

In [91]:
song_features = sp.audio_features([spotify_song_id])
song_input = pd.DataFrame(song_features).drop(columns=['analysis_url','track_href','uri','type']).set_index('id')
song_np = song_input.values
print(song_np)

[[ 3.07000e-01  6.43000e-01  1.91606e+05  7.66000e-01  1.38000e-05
   6.00000e+00  1.07000e-01 -4.78800e+00  0.00000e+00  3.84000e-01
   9.70920e+01  4.00000e+00  4.63000e-01]]


In [93]:
predictions = model.predict(song_np)
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1.0]
[[1.]]
